In [ ]:
#Module Needed: mkl,da,numpy

In [2]:
import mkl
import numpy as np
import dask.array as da

# Process an array with multiple threads

Multiple threads to process simultaneously different parts of the same array. `dask` automatically provides this feature by replacing the `numpy` function with `dask` functions. The key concept is a chunk, each chunk of data is executed separately by different threads. For example for a matrix we define a 2D block size and each of those blocks can be executed independently and then the results accumulated to get to the final answer.

### Library Dependancies

Need mkl, numpy. Install mkl with pip: ```pip install mkl```. Install numpy with pip: ```pip install numpy```.

In [3]:
# Currently numpy on some platforms is already multithreaded thanks to Intel MKL,
# for this example we disable multithreading
#import mkl
mkl.set_num_threads(1)

2

In [4]:
#import numpy as np
#import dask.array as da

In [5]:
A = np.random.rand(20000,4000)

`%whos` is a magic function provided by `IPython` that gives memory consumption of defined variables

In [6]:
%whos

Variable   Type       Data/Info
-------------------------------
A          ndarray    20000x4000: 80000000 elems, type `float64`, 640000000 bytes (610.3515625 Mb)
da         module     <module 'dask.array' from<...>/dask/array/__init__.py'>
mkl        module     <module 'mkl' from '/cm/s<...>ackages/mkl/__init__.py'>
np         module     <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>


In [7]:
A

array([[3.36534133e-01, 1.73165088e-01, 7.76990336e-02, ...,
        9.39617291e-01, 3.48124101e-01, 9.56757224e-01],
       [9.15795157e-01, 6.56350013e-01, 3.40925136e-01, ...,
        5.41135662e-01, 6.56415027e-02, 9.91193310e-01],
       [7.67082042e-01, 5.39687617e-01, 9.12233840e-01, ...,
        5.06295248e-01, 4.06398017e-01, 1.50915338e-01],
       ...,
       [5.09401604e-02, 7.03208389e-01, 9.85696543e-01, ...,
        2.49981270e-01, 1.79237166e-01, 2.16283267e-01],
       [9.66400269e-01, 9.78407049e-01, 7.92307810e-01, ...,
        6.37743445e-01, 4.52213126e-01, 9.53711454e-01],
       [5.94657111e-01, 9.83394215e-01, 8.56739734e-01, ...,
        6.44271477e-01, 2.45034927e-01, 5.63108009e-04]])

First let's perform some operations on the matrix in pure `numpy`, using a single thread

In [8]:
%time B = A**2 + np.sin(A) * A * np.log(A)

CPU times: user 3.05 s, sys: 162 ms, total: 3.21 s
Wall time: 3.22 s


## Processing with dask

First create a chunked `dask` array from the `numpy` array

In [9]:
A_dask = da.from_array(A, chunks=(2000, 1000))

In [10]:
A_dask.numblocks

(10, 4)

Then replace each function with the equivalent provided by `dask`, it implements most of the `numpy` functions and operations.

In [11]:
compute_B = (A_dask**2 + da.sin(A_dask) * A_dask * da.log(A_dask))

In [12]:
%time B_dask = compute_B.compute(num_workers=1)

CPU times: user 3.35 s, sys: 157 ms, total: 3.5 s
Wall time: 3.52 s


In [13]:
%time B_dask = compute_B.compute(num_workers=2)

CPU times: user 3.33 s, sys: 209 ms, total: 3.54 s
Wall time: 2.06 s


In [14]:
#%time B_dask = compute_B.compute(num_workers=12)

In [15]:
#%time B_dask = compute_B.compute(num_workers=num_workers)

In [16]:
assert np.allclose(B, B_dask)